# Cldn11

In [ ]:
# Import packages 
import sys
import numpy as np
import pandas as pd
import seaborn as sns
from natsort import natsorted
from scipy.io import loadmat, savemat
from skimage.filters import threshold_otsu
from skimage.color import label2rgb
from tqdm.notebook import tqdm
from skimage.feature import peak_local_max
from skimage.morphology import binary_dilation, disk
from skimage.util import regular_seeds, img_as_ubyte
from skimage.segmentation import watershed
from skimage.measure import regionprops_table

import anndata as ad
import scanpy as sc
# Customized packages 
from starmap.sequencing import *

In [ ]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42

## Path

In [ ]:
current_sample = 'ribomap'

In [ ]:
# IO path 
base_path = 'Z:/Data/Processed/2022-11-28-Hu-Cldn11/output/'
    
seg_path = os.path.join(base_path, current_sample, 'segmentation')
if not os.path.exists(seg_path):
    os.mkdir(seg_path)
    
out_path = 'Z:/Data/Analyzed/2022-09-05-Hu-Tissue/output'
# star_path = os.path.join(base_path, "rep3/protein/")
# if not os.path.exists(seg_path):
#     os.mkdir(seg_path)
    
# grid_path = os.path.join(seg_path, 'grid')
# if not os.path.exists(grid_path):
#     os.mkdir(grid_path)

## Input

In [ ]:
# load sample image 
temp = load_nissl_image(os.path.join(base_path, current_sample, 'stitched_images'), fname=f"DAPI.tif")
temp.shape

## Grid segmentation

In [ ]:
# Test
# img = np.ones((6, 6), dtype=int)
# img = img_as_ubyte(img * 255)
# seeds = regular_seeds((6, 6), 4)
# w1 = watershed(img, seeds, compactness=0.01)
# plt.imshow(w1, cmap='Greys')

0.284 micron per pixel

In [ ]:
grid_interval = 30 # micron

read_dim = (temp.shape[0] * 0.284, temp.shape[1] * 0.284)
read_area = read_dim[0] * read_dim[1]
grid_area = grid_interval**2
n_seeds = int(read_area / grid_area)
n_seeds

In [ ]:
scaled_dim = (round(temp.shape[0] * 0.25), round(temp.shape[1] * 0.25))
img = np.ones(scaled_dim, dtype=int)
img = img_as_ubyte(img * 255)
seeds = regular_seeds(scaled_dim, n_seeds)
w1 = watershed(img, seeds, compactness=0.01)

In [ ]:
plt.imshow(w1, cmap='Greys')

In [ ]:
tifffile.imsave(os.path.join(seg_path, f"{current_sample}_grid_{grid_interval}.tif"), w1.astype(np.uint16))

## Grid quantification

In [ ]:
grid_interval = 30
grid_seg = load_label_image(seg_path, fname=f"{current_sample}_grid_{grid_interval}_org.tif")

In [ ]:
# load overlay image 
raw_path = os.path.join(base_path, current_sample, 'stitched_images')

amplicon = load_nissl_image(raw_path, fname=f"amplicon.tif")
protein = load_nissl_image(raw_path, fname=f"protein.tif")
protein_mask = load_nissl_image(raw_path, fname=f"protein_mask.tif")
protein_mask = protein_mask > 0

RIBO: 0.4, STAR: 0.7

In [ ]:
# identify points 
coordinates = peak_local_max(amplicon, min_distance=3, threshold_rel=0.4)
coordinates.shape

In [ ]:
# save dot images
dots_img = np.zeros(amplicon.shape)
dots_img[coordinates[:, 0], coordinates[:, 1]] = 1
dots_img_out = binary_dilation(dots_img, disk(2))
dots_img_out = img_as_ubyte(dots_img_out)

tifffile.imsave(os.path.join(seg_path, f"{current_sample}_dots.tif"), dots_img_out)

In [ ]:
%%time
        
protein_intensity = []
protein_pixel = []
rna = []
rna_intensity = []

# Iterate through cells
print('Iterate cells...')
for i, region in enumerate(tqdm(regionprops(grid_seg))):
      # print(region.label)
    
    # protein intensity
    curr_intensity = np.sum(protein[region.coords[:, 0], region.coords[:, 1]])
    protein_intensity.append(round(curr_intensity, 2))
    
    # protein pixel
    curr_pixel = np.sum(protein_mask[region.coords[:, 0], region.coords[:, 1]])
    protein_pixel.append(round(curr_pixel, 2))
        
    # rna intensity
    curr_intensity = np.sum(amplicon[region.coords[:, 0], region.coords[:, 1]])
    rna_intensity.append(round(curr_intensity, 2))
    
    # rna
    curr_pixel = np.sum(dots_img[region.coords[:, 0], region.coords[:, 1]])
    rna.append(round(curr_pixel, 2))

In [ ]:
quantification_df = pd.DataFrame({"RNA": rna, "rna_intensity": rna_intensity, "protein_pixel": protein_pixel, "protein_intensity": protein_intensity, })
quantification_df

In [ ]:
from datetime import datetime
out_path = 'Z:/Data/Analyzed/2022-09-05-Hu-Tissue/output'
date = datetime.today().strftime('%Y-%m-%d')
quantification_df.to_csv(os.path.join(out_path, f"{date}-{current_sample}-Cldn11-grid-quantification.csv"))

## Single cell segmentation

In [ ]:
scale_path = os.path.join(base_path, current_sample, 'scaled_05')
if not os.path.exists(scale_path):
    os.mkdir(scale_path)

In [ ]:
# use fiji centroids
centroids_df = pd.read_csv(os.path.join(scale_path, 'centroid.csv'), index_col=0)
centroids = centroids_df.loc[:,[ 'Y', 'X']].values

In [ ]:
# # Load dapi label
# dapi = load_label_image(scale_path, fname='DAPI_label.tif')

# # Get cell locations 
# centroids = []

# for i, region in enumerate(tqdm(regionprops(dapi))):
#     centroids.append(region.centroid)

# centroids = np.array(centroids)

In [ ]:
# Load 2D Nissl image 
overlay = load_nissl_image(scale_path, fname="overlay.tif")

In [ ]:
%%time
# Segmentation
print("Gaussian & Thresholding")
blurred_nissl_seg = gaussian(overlay.astype(np.float), 3)
threhold = threshold_otsu(blurred_nissl_seg)
blurred_nissl_seg = blurred_nissl_seg > threhold
# blurred_nissl_seg = gaussian(nissl.astype(np.float), 10) > 50
blurred_nissl_seg = binary_dilation(blurred_nissl_seg, selem=disk(2))

print("Assigning markers")
centroids = centroids.astype(int)
markers = np.zeros(blurred_nissl_seg.shape, dtype=np.uint8)
for i in range(centroids.shape[0]):
    x, y = centroids[i, :]
    if x < blurred_nissl_seg.shape[0] and y < blurred_nissl_seg.shape[1]:
        markers[x-1, y-1] = 1
markers = ndi.label(markers)[0]

print("Watershed")
labels = watershed(blurred_nissl_seg, markers, mask=blurred_nissl_seg)
labels_line = watershed(blurred_nissl_seg, markers, mask=blurred_nissl_seg, watershed_line=True)
print(f"Labeled {len(np.unique(labels)) - 1} cells")

plt.figure(figsize=(10,20))
plt.imshow(labels_line)

print(f"Saving files to {scale_path}")
tifffile.imsave(os.path.join(scale_path, "labeled_cells_line.tif"), labels_line.astype(np.uint32))
tifffile.imsave(os.path.join(scale_path, "labeled_cells.tif"), labels.astype(np.uint32))

## Single cell quantification

In [ ]:
seg_path

In [ ]:
sc_seg = load_label_image(seg_path, fname=f"labeled_cells.tif")
sc_seg = sc_seg.astype(np.uint32)

In [ ]:
# load overlay image 
raw_path = os.path.join(base_path, current_sample, 'stitched_images')

amplicon = load_nissl_image(raw_path, fname=f"amplicon.tif")
protein = load_nissl_image(raw_path, fname=f"protein.tif")
protein_mask = load_nissl_image(raw_path, fname=f"protein_mask.tif")
protein_mask = protein_mask > 0

Grod: RIBO: 0.4, STAR: 0.7

SC: RIBO: 0.4, STAR: 0.8

In [ ]:
# identify points 
coordinates = peak_local_max(amplicon, min_distance=3, threshold_rel=0.8)
coordinates.shape

In [ ]:
# save dot images
dots_img = np.zeros(amplicon.shape)
dots_img[coordinates[:, 0], coordinates[:, 1]] = 1
# dots_img_out = binary_dilation(dots_img, disk(2))
# dots_img_out = img_as_ubyte(dots_img_out)

# tifffile.imsave(os.path.join(seg_path, f"{current_sample}_dots.tif"), dots_img_out)

In [ ]:
%%time
        
protein_intensity = []
protein_pixel = []
rna = []
row = []
column = []

# Iterate through cells
print('Iterate cells...')
for i, region in enumerate(tqdm(regionprops(sc_seg))):
      # print(region.label)
    
    # protein intensity
    curr_intensity = np.sum(protein[region.coords[:, 0], region.coords[:, 1]])
    protein_intensity.append(round(curr_intensity, 2))
    
    # protein pixel
    curr_pixel = np.sum(protein_mask[region.coords[:, 0], region.coords[:, 1]])
    protein_pixel.append(round(curr_pixel, 2))
        
    # rna
    curr_pixel = np.sum(dots_img[region.coords[:, 0], region.coords[:, 1]])
    rna.append(round(curr_pixel, 2))
    
    row.append(region.centroid[0])
    column.append(region.centroid[1])

In [ ]:
quantification_df = pd.DataFrame({"RNA": rna, "protein_intensity": protein_intensity, "protein_pixel": protein_pixel, 'row': row, 'column': column})
quantification_df

In [ ]:
from datetime import datetime
out_path = 'Z:/Data/Analyzed/2022-09-05-Hu-Tissue/output'
date = datetime.today().strftime('%Y-%m-%d')
quantification_df.to_csv(os.path.join(out_path, f"{date}-{current_sample}-Cldn11-sc-quantification.csv"))

## Analysis

### grid

In [ ]:
ribo_grid = pd.read_csv(os.path.join(out_path, '2023-01-10-ribomap-Cldn11-grid-quantification.csv'), index_col=0)
ribo_grid['RNA_norm'] = ribo_grid['RNA'] / ribo_grid['RNA'].max()
ribo_grid['rna_intensity_norm'] = ribo_grid['rna_intensity'] / ribo_grid['rna_intensity'].max()
ribo_grid['protein_intensity_norm'] = ribo_grid['protein_intensity'] / ribo_grid['protein_intensity'].max()
ribo_grid['protein_pixel_norm'] = ribo_grid['protein_pixel'] / ribo_grid['protein_pixel'].max()
ribo_grid

In [ ]:
star_grid = pd.read_csv(os.path.join(out_path, '2023-01-10-starmap-Cldn11-grid-quantification.csv'), index_col=0)
star_grid['RNA_norm'] = star_grid['RNA'] / star_grid['RNA'].max()
star_grid['rna_intensity_norm'] = star_grid['rna_intensity'] / star_grid['rna_intensity'].max()
star_grid['protein_intensity_norm'] = star_grid['protein_intensity'] / star_grid['protein_intensity'].max()
star_grid['protein_pixel_norm'] = star_grid['protein_pixel'] / star_grid['protein_pixel'].max()
star_grid

In [ ]:
grid_interval = 30
ribo_grid_seg = load_label_image(os.path.join(base_path, 'ribomap', 'segmentation'), fname=f"ribomap_grid_{grid_interval}_org.tif")
star_grid_seg = load_label_image(os.path.join(base_path, 'starmap', 'segmentation'), fname=f"starmap_grid_{grid_interval}_org.tif")

In [ ]:
print(np.unique(ribo_grid_seg[:, 0]).shape, np.unique(ribo_grid_seg[0, :]).shape)

In [ ]:
fig, axs = plt.subplots(figsize=(33, 13), ncols=3)
ribo_grid_img = np.reshape(ribo_grid['rna_intensity_norm'].values, (243, 178)) # rna_intensity_norm
sns.heatmap(ribo_grid_img, cmap='jet', ax=axs[0])
ribo_grid_img = np.reshape(ribo_grid['protein_pixel_norm'].values, (243, 178))
sns.heatmap(ribo_grid_img, cmap='jet', ax=axs[1])
ribo_grid_img = np.reshape(ribo_grid['protein_intensity_norm'].values, (243, 178))
sns.heatmap(ribo_grid_img, cmap='jet', ax=axs[2])
axs[0].set_title('RNA')
axs[1].set_title('protein_pixel')
axs[2].set_title('protein_intensity')

axs[0].axis('off')
axs[1].axis('off')
axs[2].axis('off')
fig.suptitle('RIBOmap')
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(11, 13))
ribo_grid_img = np.reshape(ribo_grid['protein_intensity_norm'].values, (243, 178))
sns.heatmap(ribo_grid_img, cmap='viridis', vmin=0, vmax=0.85)

axs.axis('off')
plt.savefig(os.path.join(fig_path, 'heatmap_ribomap_cldn11_grid.pdf'))
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(33, 13), ncols=3)
star_grid_img = np.reshape(star_grid['rna_intensity_norm'].values, (246, 190)) # rna_intensity_norm
sns.heatmap(star_grid_img, cmap='jet', ax=axs[0])
star_grid_img = np.reshape(star_grid['protein_pixel_norm'].values, (246, 190))
sns.heatmap(star_grid_img, cmap='jet', ax=axs[1])
star_grid_img = np.reshape(star_grid['protein_intensity_norm'].values, (246, 190))
sns.heatmap(star_grid_img, cmap='jet', ax=axs[2])
axs[0].set_title('RNA')
axs[1].set_title('protein_pixel')
axs[2].set_title('protein_intensity')

axs[0].axis('off')
axs[1].axis('off')
axs[2].axis('off')
fig.suptitle('STARmap')
plt.show()

In [ ]:
fig_path = 'Z:/Data/Analyzed/2022-09-05-Hu-Tissue/figures'

In [ ]:
# def reject_outliers(sr, iq_range=0.8):
#     pcnt = (1 - iq_range) / 2
#     qlow, median, qhigh = sr.dropna().quantile([pcnt, 0.50, 1-pcnt])
#     iqr = qhigh - qlow
#     return sr[ (sr - median).abs() <= iqr]

In [ ]:
rna_slot = 'rna_intensity_norm' # RNA
protein_slot = 'protein_intensity_norm' # protein_pixel

ribo_grid_filtered = ribo_grid.copy()
star_grid_filtered = star_grid.copy()

ribo_protein_quantile = ribo_grid_filtered['protein_intensity_norm'].quantile(0.999)
ribo_rna_quantile = ribo_grid_filtered['rna_intensity_norm'].quantile(0.999)
ribo_grid_filtered = ribo_grid_filtered.loc[(ribo_grid_filtered['protein_intensity_norm'] < ribo_protein_quantile) & (ribo_grid_filtered['rna_intensity_norm'] < ribo_rna_quantile), :]

star_protein_quantile = star_grid_filtered['protein_intensity_norm'].quantile(0.999)
star_rna_quantile = star_grid_filtered['rna_intensity_norm'].quantile(0.999)
star_grid_filtered = star_grid_filtered.loc[(star_grid_filtered['protein_intensity_norm'] < star_protein_quantile) & (star_grid_filtered['rna_intensity_norm'] < star_rna_quantile), :]

ribo_grid_filtered['rna_intensity_norm'] = ribo_grid_filtered['rna_intensity'] / ribo_grid_filtered['rna_intensity'].max()
star_grid_filtered['rna_intensity_norm'] = star_grid_filtered['rna_intensity'] / star_grid_filtered['rna_intensity'].max()


ribo_grid_filtered['protein_intensity_norm'] = ribo_grid_filtered['protein_intensity'] / ribo_grid_filtered['protein_intensity'].max()
star_grid_filtered['protein_intensity_norm'] = star_grid_filtered['protein_intensity'] / star_grid_filtered['protein_intensity'].max()

# ribo_grid_filtered = ribo_grid.loc[(ribo_grid[rna_slot] != 0) & (ribo_grid[protein_slot] != 0)].copy()
# star_grid_filtered = star_grid.loc[(star_grid[rna_slot] != 0) & (star_grid[protein_slot] != 0)].copy()

ribo_protein_vector = ribo_grid_filtered[protein_slot].values
star_protein_vector = star_grid_filtered[protein_slot].values
ribo_expr_vector = ribo_grid_filtered[rna_slot].values
star_expr_vector = star_grid_filtered[rna_slot].values

# ribo_protein_vector = reject_outliers(ribo_grid_filtered[protein_slot]).values
# star_protein_vector = reject_outliers(star_grid_filtered[protein_slot]).values
# ribo_expr_vector = reject_outliers(ribo_grid_filtered[rna_slot]).values
# star_expr_vector = reject_outliers(star_grid_filtered[rna_slot]).values

# ribo_protein_vector = np.log1p(ribo_protein_vector)
# star_protein_vector = np.log1p(star_protein_vector)
# ribo_expr_vector = np.log1p(ribo_expr_vector)
# star_expr_vector = np.log1p(star_expr_vector)

from scipy import stats
ribo_p_corr = stats.pearsonr(ribo_protein_vector, ribo_expr_vector)
ribo_corre_df = pd.DataFrame({'Cldn11_protein': ribo_protein_vector, 'RIBOmap': ribo_expr_vector})

star_p_corr = stats.pearsonr(star_protein_vector, star_expr_vector)
star_corre_df = pd.DataFrame({'Cldn11_protein': star_protein_vector, 'STARmap': star_expr_vector})

fig, axs = plt.subplots(figsize=(10, 5), ncols=2)
g1 = sns.regplot(x='Cldn11_protein', y='RIBOmap', data=ribo_corre_df, scatter_kws={'s': 1}, line_kws={'color': 'r'}, ax=axs[0])
# axs[0].set_axis_labels('Gfap_protein', 'RIBOmap')
# g1.set(ylim=(None, 0.85))
# g1.set(xlim=(0, 0.8))
axs[0].set_title(f"Pearson's correlation coefficient: {round(ribo_p_corr[0], 3)}")

g2 = sns.regplot(x='Cldn11_protein', y='STARmap', data=star_corre_df, scatter_kws={'s': 1}, line_kws={'color': 'r'}, ax=axs[1])
# g2.set(ylim=(None, 0.85))
# g2.set_axis_labels('Gfap_protein', 'STARmap')
axs[1].set_title(f"Pearson's correlation coefficient: {round(star_p_corr[0], 3)}")

plt.savefig(os.path.join(fig_path, '2023-01-11-correlation_rna_protein_cldn11_grid.pdf'))
plt.show()

### single cell

In [ ]:
ribo_grid = pd.read_csv(os.path.join(out_path, '2022-12-17-ribomap-Cldn11-sc-quantification.csv'), index_col=0)
ribo_grid['RNA_norm'] = ribo_grid['RNA'] / ribo_grid['RNA'].max()
ribo_grid['protein_intensity_norm'] = ribo_grid['protein_intensity'] / ribo_grid['protein_intensity'].max()
ribo_grid['protein_pixel_norm'] = ribo_grid['protein_pixel'] / ribo_grid['protein_pixel'].max()
ribo_grid

In [ ]:
star_grid = pd.read_csv(os.path.join(out_path, '2022-12-17-starmap-Cldn11-sc-quantification.csv'), index_col=0)
star_grid['RNA_norm'] = star_grid['RNA'] / star_grid['RNA'].max()
star_grid['protein_intensity_norm'] = star_grid['protein_intensity'] / star_grid['protein_intensity'].max()
star_grid['protein_pixel_norm'] = star_grid['protein_pixel'] / star_grid['protein_pixel'].max()
star_grid

In [ ]:
fig, axs = plt.subplots(figsize=(30, 15), ncols=3)
g1 = sns.scatterplot(x='column', y='row', hue='RNA', palette='rainbow', s=10, data=ribo_grid, ax=axs[0])
g1.set_title('RNA')
g1.invert_yaxis()
g1.axes.xaxis.set_visible(False)
g1.axes.yaxis.set_visible(False)

g2 = sns.scatterplot(x='column', y='row', hue='protein_intensity', palette='rainbow', s=10, data=ribo_grid, ax=axs[1])
g2.set_title('protein_intensity')
g2.invert_yaxis()
g2.axes.xaxis.set_visible(False)
g2.axes.yaxis.set_visible(False)

g3 = sns.scatterplot(x='column', y='row', hue='protein_pixel', palette='rainbow', s=10, data=ribo_grid, ax=axs[2])
g3.set_title('protein_pixel')
g3.invert_yaxis()
g3.axes.xaxis.set_visible(False)
g3.axes.yaxis.set_visible(False)
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(30, 15), ncols=3)
g1 = sns.scatterplot(x='column', y='row', hue='RNA', palette='rainbow', s=10, data=star_grid, ax=axs[0])
g1.set_title('RNA')
g1.invert_yaxis()
g1.axes.xaxis.set_visible(False)
g1.axes.yaxis.set_visible(False)

g2 = sns.scatterplot(x='column', y='row', hue='protein_intensity', palette='rainbow', s=10, data=star_grid, ax=axs[1])
g2.set_title('protein_intensity')
g2.invert_yaxis()
g2.axes.xaxis.set_visible(False)
g2.axes.yaxis.set_visible(False)

g3 = sns.scatterplot(x='column', y='row', hue='protein_pixel', palette='rainbow', s=10, data=star_grid, ax=axs[2])
g3.set_title('protein_pixel')
g3.invert_yaxis()
g3.axes.xaxis.set_visible(False)
g3.axes.yaxis.set_visible(False)
plt.show()

In [ ]:
# astro
rna_slot = 'RNA'
protein_slot = 'protein_pixel'

ribo_grid_filtered = ribo_grid.copy()
star_grid_filtered = star_grid.copy()

# ribo_grid_filtered = ribo_grid.loc[(ribo_grid[rna_slot] != 0) & (ribo_grid[protein_slot] != 0)].copy()
# star_grid_filtered = star_grid.loc[(star_grid[rna_slot] != 0) & (star_grid[protein_slot] != 0)].copy()

ribo_protein_vector = ribo_grid_filtered[protein_slot].values
star_protein_vector = star_grid_filtered[protein_slot].values
ribo_expr_vector = ribo_grid_filtered[rna_slot].values
star_expr_vector = star_grid_filtered[rna_slot].values

# ribo_protein_vector = np.log1p(ribo_protein_vector)
# star_protein_vector = np.log1p(star_protein_vector)
# ribo_expr_vector = np.log1p(ribo_expr_vector)
# star_expr_vector = np.log1p(star_expr_vector)

from scipy import stats
ribo_p_corr = stats.pearsonr(ribo_protein_vector, ribo_expr_vector)
ribo_corre_df = pd.DataFrame({'Cldn11_protein': ribo_protein_vector, 'RIBOmap': ribo_expr_vector})

star_p_corr = stats.pearsonr(star_protein_vector, star_expr_vector)
star_corre_df = pd.DataFrame({'Cldn11_protein': star_protein_vector, 'STARmap': star_expr_vector})

fig, axs = plt.subplots(figsize=(10, 5), ncols=2)
g1 = sns.regplot(x='Cldn11_protein', y='RIBOmap', data=ribo_corre_df, scatter_kws={'s': 1}, line_kws={'color': 'r'}, ax=axs[0])
# axs[0].set_axis_labels('Gfap_protein', 'RIBOmap')
axs[0].set_title(f"Pearson's correlation coefficient: {round(ribo_p_corr[0], 3)}")

g2 = sns.regplot(x='Cldn11_protein', y='STARmap', data=star_corre_df, scatter_kws={'s': 1}, line_kws={'color': 'r'}, ax=axs[1])
# g2.set_axis_labels('Gfap_protein', 'STARmap')
axs[1].set_title(f"Pearson's correlation coefficient: {round(star_p_corr[0], 3)}")

plt.show()

### manual

In [ ]:
seg_path

In [ ]:
# load segmentation image 
seg = load_nissl_image(seg_path, fname=f"ccf_zf_labels_org.tif")
seg.shape

In [ ]:
%%time

labels = []
protein_intensity = []
areas = []
# protein_pixel = []
rna = []

# Iterate through cells
print('Iterate cells...')
for i, region in enumerate(tqdm(regionprops(seg))):
      # print(region.label)
    labels.append(region.label)
    areas.append(region.area)
    
    # protein intensity
    curr_intensity = np.sum(protein[region.coords[:, 0], region.coords[:, 1]])
    protein_intensity.append(round(curr_intensity, 2))
    
#     # protein pixel
#     curr_pixel = np.sum(protein_mask[region.coords[:, 0], region.coords[:, 1]])
#     protein_pixel.append(round(curr_pixel, 2))
        
    # rna
    curr_pixel = np.sum(amplicon[region.coords[:, 0], region.coords[:, 1]])
    rna.append(round(curr_pixel, 2))

sns.histplot(areas)

In [ ]:
quantification_df = pd.DataFrame({"region_label": labels, "area": areas,  "protein_intensity": protein_intensity, 'rna_intensity': rna})
quantification_df['protein_intensity_norm'] = quantification_df['protein_intensity'] / quantification_df['area']
quantification_df['protein_intensity_norm'] = quantification_df['protein_intensity_norm'] / quantification_df['protein_intensity_norm'].max()
quantification_df['rna_intensity_norm'] = quantification_df['rna_intensity'] / quantification_df['area']
quantification_df

In [ ]:
# map region
label_dict = {1: 'Isocortex', 2: 'HIP', 3: 'TH', 4: 'HY', 
             5: 'Fiber tracts', 6: 'CNU', 7: 'CTXsp', 8: 'OLF'}

In [ ]:
quantification_df['region_name'] = quantification_df['region_label']
quantification_df['region_name'] = quantification_df['region_name'].map(label_dict)
quantification_df

In [ ]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
df_scaled = std_scaler.fit_transform(quantification_df.protein_intensity_norm.to_numpy().reshape([8, 1]))
# df_scaled = pd.DataFrame(df_scaled, columns=df_te.index, index=df_te.columns)
quantification_df['protein_intensity_scaled'] = df_scaled
quantification_df

In [ ]:
plt.imshow(protein)

In [ ]:
seg_scaled = seg.copy()
seg_scaled = seg_scaled.astype('float')

for i, current_label in enumerate(quantification_df['region_label']):
    print(current_label)
    seg_scaled[np.where(seg == current_label)] = quantification_df.loc[i, 'protein_intensity_norm']

In [ ]:
vmin = quantification_df['protein_intensity_norm'].min()

In [ ]:
fig, ax = plt.subplots(figsize=(28, 21))
plt.imshow(seg_scaled, cmap='viridis', vmin=vmin)
plt.axis('off')
plt.colorbar()
plt.savefig(os.path.join(fig_path, 'heatmap_Cldn11_quantification_vmin.pdf'))

In [ ]:
from datetime import datetime
out_path = 'Z:/Data/Analyzed/2022-09-05-Hu-Tissue/output'
date = datetime.today().strftime('%Y-%m-%d')
quantification_df.to_csv(os.path.join(out_path, f"{date}-Cldn11-protein-quantification.csv"))